In [2]:
import sys
sys.path.append('..')
from polynomial_program import PolynomialProgram

In [6]:
from qiskit import Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import I, X, Y, Z
from qiskit.aqua.operators.list_ops import SummedOp
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.circuit.library import RealAmplitudes
from qiskit.visualization import circuit_visualization
from docplex.mp.model import Model
from qiskit.optimization.applications.ising import docplex
# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np

from collections import OrderedDict

In [3]:
M = [2,6,3]
K = [1,4,2]
T = [12, 6, 24]
d = 19

def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m

time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [4]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0

optimal_key = "0000111011"

def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0
     
    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1
            
    print('most likely solution: ',sample_most_likely(dict_res))      
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)

    
def is_correct(key):
    return solution_vector_correct(key) and execution_time(key) == d


correct_machines = ['00', '01', '11']
machine_to_index = {'00': 2, '01': 1, '11': 0}

def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    return task1_machine in correct_machines \
        and task2_machine in correct_machines \
        and task3_machine in correct_machines


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_time = time_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_time = time_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_time = time_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1

    return task1_time + task2_time + task3_time + slack_sum


def execution_cost(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_cost = cost_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_cost = cost_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_cost = cost_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    return task1_cost + task2_cost + task3_cost


def incorrect_machine_count(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])


    return (0 if k[0:2] in correct_machines else 1) \
         + (0 if k[2:4] in correct_machines else 1) \
         + (0 if k[4:6] in correct_machines else 1)

In [5]:
def get_cost_model(x):
    return sum([
            cost_matrix[0, i] * x[2 * i]
            + cost_matrix[1, i] * (x[2 * i + 1] - x[2 * i]) ** 2
            + cost_matrix[2, i] * (1 - x[2 * i + 1])
            for i in range(0, 3)
    ])


def get_machine_usage_model(x):
    return sum([
        x[2 * i] - x[2 * i] * x[2 * i + 1] for i in range(0, 3)
    ])


def get_deadline_model(x):
    time_sum = sum([
            time_matrix[0, i] * x[2 * i]
            + time_matrix[1, i] * (x[2 * i + 1] - x[2 * i]) ** 2
            + time_matrix[2, i] * (1 - x[2 * i + 1])
                for i in range(0, 3)
    ])
    slack_sum = 8 * x[6] + 4 * x[7] + 2 * x[8] + x[9]
    time_constraint = (d - time_sum - slack_sum) ** 2
    
    return time_constraint

In [6]:
def compute_eigenvalues(quibit_op):
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 1024
    eigensolver = NumPyEigensolver(qubit_op, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print('state\t\ttime\tcost\tmachine use\tcorrect\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenstate = eigenstate[::-1]
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{execution_time(eigenstate)}\t{execution_cost(eigenstate)}', end='')
        print(f'\t{incorrect_machine_count(eigenstate)}\t\t{is_correct(eigenstate)}\t{eigenvalue}')

# Three objectives

In [7]:
A = 1
B = 20
C = 10

In [8]:
pp = PolynomialProgram(10)

pp.add_objective(get_cost_model(pp.x), A)
pp.add_objective(get_machine_usage_model(pp.x), B)
pp.add_objective(get_deadline_model(pp.x), C)

In [9]:
qubit_op, offset = pp.to_ising()

In [10]:
qubit_op, offset = pp.to_ising()

state		time	cost	machine use	correct	eigenvalue
1101110000	19.0	22.0	0		True	(-377.5+0j)
0011110000	19.0	23.0	0		True	(-376.5+0j)
0111110010	19.0	23.0	0		True	(-376.5+0j)
0001110010	19.0	24.0	0		True	(-375.5+0j)
0100110011	19.0	24.0	0		True	(-375.5+0j)
0101110100	19.0	24.0	0		True	(-375.5+0j)
0000110001	19.0	24.0	0		True	(-375.5+0j)
1111000010	19.0	25.0	0		True	(-374.5+0j)
1111010110	19.0	25.0	0		True	(-374.5+0j)
1101000100	19.0	26.0	0		True	(-373.5+0j)
1100000011	19.0	26.0	0		True	(-373.5+0j)
1101011000	19.0	26.0	0		True	(-373.5+0j)
1100010111	19.0	26.0	0		True	(-373.5+0j)
0011000100	19.0	27.0	0		True	(-372.5+0j)
0111000110	19.0	27.0	0		True	(-372.5+0j)
0011011000	19.0	27.0	0		True	(-372.5+0j)
0111011010	19.0	27.0	0		True	(-372.5+0j)
0001000110	19.0	28.0	0		True	(-371.5+0j)
0100011011	19.0	28.0	0		True	(-371.5+0j)
0000000101	19.0	28.0	0		True	(-371.5+0j)
0101001000	19.0	28.0	0		True	(-371.5+0j)
0000011001	19.0	28.0	0		True	(-371.5+0j)
0100000111	19.0	28.0	0		True	(-371.5+0j)
010101110

1100000111	23.0	26.0	0		False	(-213.5+0j)
1101001000	23.0	26.0	0		False	(-213.5+0j)
1101000000	15.0	26.0	0		False	(-213.5+0j)
1100011011	23.0	26.0	0		False	(-213.5+0j)
1101011100	23.0	26.0	0		False	(-213.5+0j)
1101010100	15.0	26.0	0		False	(-213.5+0j)
0111010110	15.0	27.0	0		False	(-212.5+0j)
0111001010	23.0	27.0	0		False	(-212.5+0j)
0111011110	23.0	27.0	0		False	(-212.5+0j)
0011001000	23.0	27.0	0		False	(-212.5+0j)
0111000010	15.0	27.0	0		False	(-212.5+0j)
0011010100	15.0	27.0	0		False	(-212.5+0j)
0011011100	23.0	27.0	0		False	(-212.5+0j)
0011000000	15.0	27.0	0		False	(-212.5+0j)
0100001011	23.0	28.0	0		False	(-211.5+0j)
0101001100	23.0	28.0	0		False	(-211.5+0j)
0000010101	15.0	28.0	0		False	(-211.5+0j)
0000001001	23.0	28.0	0		False	(-211.5+0j)
0101000100	15.0	28.0	0		False	(-211.5+0j)
0001011110	23.0	28.0	0		False	(-211.5+0j)
0000000001	15.0	28.0	0		False	(-211.5+0j)
0100011111	23.0	28.0	0		False	(-211.5+0j)
0101011000	15.0	28.0	0		False	(-211.5+0j)
0100000011	15.0	28.0	0		False	(-21

1110001000	22.0	22.0	1		False	(463.5+0j)
1110011100	22.0	22.0	1		False	(463.5+0j)
0010001010	22.0	24.0	1		False	(465.5+0j)
0010011110	22.0	24.0	1		False	(465.5+0j)
0110001100	22.0	24.0	1		False	(465.5+0j)
1011110001	16.0	15.0	1		False	(467.5+0j)
1000110010	16.0	16.0	1		False	(468.5+0j)
1001110011	16.0	16.0	1		False	(468.5+0j)
1011011001	16.0	19.0	1		False	(471.5+0j)
1011000101	16.0	19.0	1		False	(471.5+0j)
1001011011	16.0	20.0	1		False	(472.5+0j)
1000011010	16.0	20.0	1		False	(472.5+0j)
1001000111	16.0	20.0	1		False	(472.5+0j)
1000000110	16.0	20.0	1		False	(472.5+0j)
0101100001	4.0	12.0	1		False	(486.5+0j)
0100100000	4.0	12.0	1		False	(486.5+0j)
1010010110	10.0	16.0	2		False	(499.5+0j)
1010000010	10.0	16.0	2		False	(499.5+0j)
1111111000	29.0	21.0	0		False	(621.5+0j)
1100111001	29.0	22.0	0		False	(622.5+0j)
1101111010	29.0	22.0	0		False	(622.5+0j)
0011111010	29.0	23.0	0		False	(623.5+0j)
0111111100	29.0	23.0	0		False	(623.5+0j)
0001111100	29.0	24.0	0		False	(624.5+0j)
0000111011	29.0	24

1001001101	22.0	20.0	1		False	(1912.5+0j)
1000001100	22.0	20.0	1		False	(1912.5+0j)
1111100001	10.0	9.0	1		False	(1923.5+0j)
1101100011	10.0	10.0	1		False	(1924.5+0j)
1100100010	10.0	10.0	1		False	(1924.5+0j)
0111100101	10.0	11.0	1		False	(1925.5+0j)
0011100011	10.0	11.0	1		False	(1925.5+0j)
0100100110	10.0	12.0	1		False	(1926.5+0j)
0001100101	10.0	12.0	1		False	(1926.5+0j)
0000100100	10.0	12.0	1		False	(1926.5+0j)
0101100111	10.0	12.0	1		False	(1926.5+0j)
1010110100	16.0	12.0	2		False	(1935.5+0j)
1010001000	16.0	16.0	2		False	(1939.5+0j)
1010011100	16.0	16.0	2		False	(1939.5+0j)
0010100000	4.0	8.0	2		False	(1953.5+0j)
0110100010	4.0	8.0	2		False	(1953.5+0j)
1111111110	35.0	21.0	0		False	(2181.5+0j)
1100111111	35.0	22.0	0		False	(2182.5+0j)
1110111011	29.0	18.0	1		False	(2209.5+0j)
0110111111	29.0	20.0	1		False	(2211.5+0j)
0010111101	29.0	20.0	1		False	(2211.5+0j)
1110001111	29.0	22.0	1		False	(2213.5+0j)
1011111000	23.0	15.0	1		False	(2217.5+0j)
1001111010	23.0	16.0	1		False	(2218.5+0

1000100101	7.0	4.0	2		False	(5470.5+0j)
1010100001	1	0	3		False	(5497.5+0j)
1111101011	20.0	9.0	1		False	(5923.5+0j)
1101101101	20.0	10.0	1		False	(5924.5+0j)
1100101100	20.0	10.0	1		False	(5924.5+0j)
0011101101	20.0	11.0	1		False	(5925.5+0j)
0111101111	20.0	11.0	1		False	(5925.5+0j)
0000101110	20.0	12.0	1		False	(5926.5+0j)
0001101111	20.0	12.0	1		False	(5926.5+0j)
1010111110	26.0	12.0	2		False	(5935.5+0j)
1110101000	14.0	6.0	2		False	(5951.5+0j)
0010101010	14.0	8.0	2		False	(5953.5+0j)
0110101100	14.0	8.0	2		False	(5953.5+0j)
1011100101	8.0	3.0	2		False	(5959.5+0j)
1001100111	8.0	4.0	2		False	(5960.5+0j)
1000100110	8.0	4.0	2		False	(5960.5+0j)
1010100010	2	0	3		False	(5987.5+0j)
1111101100	21.0	9.0	1		False	(6433.5+0j)
1101101110	21.0	10.0	1		False	(6434.5+0j)
1100101101	21.0	10.0	1		False	(6434.5+0j)
0011101110	21.0	11.0	1		False	(6435.5+0j)
0000101111	21.0	12.0	1		False	(6436.5+0j)
1010111111	27.0	12.0	2		False	(6445.5+0j)
1110101001	15.0	6.0	2		False	(6461.5+0j)
0110101101	15.0	8.

In [11]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [7]:
mixer = SummedOp([
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ X,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ Z ^ X,

    I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ X ^ Z,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ X ^ I,

  
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ X    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ Z ^ X    ^ I ^ I,

    I ^ I ^ I ^ I    ^ I ^ I    ^ X ^ Z    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ X ^ I    ^ I ^ I,

    
  - I ^ I ^ I ^ I    ^ I ^ X    ^ I ^ I    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ Z ^ X    ^ I ^ I    ^ I ^ I,

    I ^ I ^ I ^ I    ^ X ^ Z    ^ I ^ I    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ X ^ I    ^ I ^ I    ^ I ^ I,

    
    I ^ I ^ I ^ X    ^ I ^ I    ^ I ^ I    ^ I ^ I,
    I ^ I ^ X ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ I,
    I ^ X ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ I,
    X ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ I,
])

In [ ]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, p=1, mixer=mixer)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

# Two objectives

In [8]:
A = 1
C = 10

In [ ]:
pp = PolynomialProgram(10)

pp.add_objective(get_cost_model(pp.x), A)
pp.add_objective(get_deadline_model(pp.x), C)

In [ ]:
qubit_op, offset = pp.to_ising()

In [ ]:
qubit_op, offset = pp.to_ising()

In [ ]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [ ]:
mixer = SummedOp([
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ X,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ Z ^ X,

    I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ X ^ Z,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ X ^ I,

  
  - I ^ I ^ I ^ I    ^ I ^ I    ^ I ^ X    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ Z ^ X    ^ I ^ I,

    I ^ I ^ I ^ I    ^ I ^ I    ^ X ^ Z    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ I ^ I    ^ X ^ I    ^ I ^ I,

    
  - I ^ I ^ I ^ I    ^ I ^ X    ^ I ^ I    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ Z ^ X    ^ I ^ I    ^ I ^ I,

    I ^ I ^ I ^ I    ^ X ^ Z    ^ I ^ I    ^ I ^ I,
  - I ^ I ^ I ^ I    ^ X ^ I    ^ I ^ I    ^ I ^ I,

    
    I ^ I ^ I ^ X    ^ I ^ I    ^ I ^ I    ^ I ^ I,
    I ^ I ^ X ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ I,
    I ^ X ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ I,
    X ^ I ^ I ^ I    ^ I ^ I    ^ I ^ I    ^ I ^ I,
])

In [ ]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, p=1, mixer=mixer)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)